<a href="https://colab.research.google.com/github/kimberly-kao/CS4650-sPacyPractice/blob/main/Hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!pip install newsapi-python



In [ ]:
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
from newsapi import NewsApiClient
import spacy

API_KEY = "9d23ebd5b162489f9eddc9f92ad4828b"
nlp_eng = spacy.load('en')
newsapi = NewsApiClient (API_KEY)

In [ ]:
pos_tag = ["VERB", "NOUN", "PROPN"]

def get_keywords_eng(doc):
  result = []
  for token in doc:
    if (token.text in nlp_eng.Defaults.stop_words or token.is_punct):
      continue
    if (token.pos_ in pos_tag):
      result.append(token.text)
  return result

In [ ]:
import pickle
import pandas as pd
from collections import Counter

articles = []

for x in range(5):
  x = x+1
  articles.append(newsapi.get_everything(q='coronavirus', language='en', 
                              from_param='2022-02-25', to='2022-03-22', 
                              sort_by='relevancy', page=x))

filename = 'articlesCOVID.pckl'
pickle.dump(articles, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
filepath = '/articlesCOVID.pckl'
pickle.dump(loaded_model, open(filepath, 'wb'))

dados = []

for i, article in enumerate(articles):
    for x in article['articles']:
        title = x['title']
        description = x['description']
        content = x['content']
        date = x['publishedAt']
        dados.append({'title':title, 'date':date, 'desc':description, 'content':content})

print("DATA EXTRACTED: ")
for x in dados:
  print(x)
print(" ")

df = pd.DataFrame(dados)
df = df.dropna()
df.head()

print("DATAFRAME AFTER DATA EXTRACTION")
print(df)
print(" ")

results = []

print("KEYWORDS FOR EACH ARTICLE: ")
for content in df.content.values:
    content = nlp_eng(content)
    print(get_keywords_eng(content))
    results.append([('#' + x[0]) for x in Counter(get_keywords_eng(content)).most_common(5)])
df['keywords'] = results
print(" ")

print("FINAL DATAFRAME: ")
print(df)


DATA EXTRACTED: 
{'title': 'Rabbi Chaim Kanievsky, Venerated Talmudic Scholar, Dies at 94', 'date': '2022-03-18T20:26:55Z', 'desc': 'He initially insisted, “Canceling Torah study is more dangerous than the coronavirus.” But he had a change of heart, even before testing positive.', 'content': 'Most significant was his ruling that instances of sexual abuse of children should be reported to the civic authorities and not handled exclusively within the Haredi community. This was a sharp depart… [+1849 chars]'}
{'title': 'Can Eric Adams Cheerlead New Yorkers Past the Pandemic?', 'date': '2022-03-20T16:11:17Z', 'desc': 'The mayor is singularly focused on bringing back New York City’s economy, now that coronavirus cases have dropped.', 'content': 'Mr. Adams, a Democrat, has had regular conversations with some of the citys most influential business leaders, including David Solomon, chief executive at the banking firm Goldman Sachs, and Jonathan… [+1705 chars]'}
{'title': 'Preschoolers Can Shed 